In [ ]:
import pandas as pd 
import numpy as np 
import xmltodict
import base64
import struct
import argparse
import os
import sys
from tqdm import tqdm
from scipy import stats

args = {
    'in_XML' : '/workspace/data/NAS/Muse2013-19ECGs_XML', # where the XML files are located, put none if these are already calculated
    'in_csv': '/workspace/data/NAS/Cedars EKG Analysis/Muse2008-19 XML Metadata.csv',# Where the csv is located for your input dataset
    'in_dataset': '/workspace/data/drives/Internal_SSD/sdc/EKG Test/',# Where the converted XMLs will be moved, or, where the numpy arrays are currently stored
    'out_dataset': '/workspace/data/drives/Internal_SSD/sdc/Comparison/', # Where the normalized EKGs are stored
    'sample':100000, # sample size to estimate mean and standard deviation
    'mean':np.array([-0.753910531911661,-0.5609376530271284,0.19297287888453685,0.6574240924693946,-0.09648643944226842,0.09648643944226842,-0.9398182103547104,-0.8866948773251518,-0.9585726095365399,-0.9142084935751398,-0.9573448180888456,-0.9300810636208064]),
    'std':np.array([32.082092358503644,34.97862852596865,38.153409045189754,27.612712586528637,19.076704522594877,19.076704522594877,42.77931881050877,63.872623440588406,61.15731462396783,54.12879139607189,48.435274440820855,43.34056377213695])
    
}# replace mean and std with none to calculate your own

In [ ]:




def file_path(path):
    return os.listdir(path)
    
#need to update this function to check the output directory for the output file and then only on newly added EKGs
#add timestamp to start file string 
#this is annoying because the XML file name is a random timestamp and the output file is the UniqueECGID


#if not os.path.exists('/D:/Muse2019ECGs_npy/'):
#    os.mkdir('/D:/Muse2019ECGs_npy/')

# parser = argparse.ArgumentParser(description='Input and outputs for XML EKG parsing')
# parser.add_argument('input', type=str)
# parser.set_defaults(output=os.getcwd() + '/EchoNet_ECG_waveforms/') #ensure this directory already exists

# args = parser.parse_args()



def decode_ekg_muse(raw_wave):
    """
    Ingest the base64 encoded waveforms and transform to numeric
    """
    # covert the waveform from base64 to byte array
    arr = base64.b64decode(bytes(raw_wave, 'utf-8'))

    # unpack every 2 bytes, little endian (16 bit encoding)
    unpack_symbols = ''.join([char*int(len(arr)/2) for char in 'h'])
    byte_array = struct.unpack(unpack_symbols,  arr)
    return byte_array


def decode_ekg_muse_to_array(raw_wave, downsample = 1):
    """
    Ingest the base64 encoded waveforms and transform to numeric
    downsample: 0.5 takes every other value in the array. Muse samples at 500/s and the sample model requires 250/s. So take every other.
    """
    try:
        dwnsmpl = int(1//downsample)
    except ZeroDivisionError:
        print("You must downsample by more than 0")
    # covert the waveform from base64 to byte array
    arr = base64.b64decode(bytes(raw_wave, 'utf-8'))

    # unpack every 2 bytes, little endian (16 bit encoding)
    unpack_symbols = ''.join([char*int(len(arr)/2) for char in 'h'])
    byte_array = struct.unpack(unpack_symbols,  arr)
    return np.array(byte_array)[::dwnsmpl]



def xml_to_np_array_file(path_to_xml, path_to_output = os.getcwd(),df=None):
    with open(path_to_xml, 'rb') as fd:
        dic = xmltodict.parse(fd.read().decode('utf8'))
    
    """
    Upload the ECG as numpy array with shape=[5000,12] ([time, leads, 1]).
    The voltage unit should be in 1 mv/unit and the sampling rate should be 250/second (total 10 second).
    The leads should be ordered as follow I, II, III, aVR, aVL, aVF, V1, V2, V3, V4, V5, V6.
    """
    try:
        pt_id = dic['RestingECG']['PatientDemographics']['PatientID']
    except:
        pt_id = "none"
    try:
        PharmaUniqueECGID = dic['RestingECG']['PharmaData']['PharmaUniqueECGID']
    except:
        PharmaUniqueECGID = "none"
    try:
        AcquisitionDateTime = dic['RestingECG']['TestDemographics']['AcquisitionDate'] + "_" + dic['RestingECG']['TestDemographics']['AcquisitionTime'].replace(":","-")
    except:
        AcquisitionDateTime = "none"    

    # try:
    #     requisition_number = dic['RestingECG']['Order']['RequisitionNumber']
    # except:
    #     print("no requisition_number")
    #     requisition_number = "none"

    #need to instantiate leads in the proper order for the model
    lead_order = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']

    """
    Each EKG will have this data structure:
    lead_data = {
        'I': np.array
    }
    """

    lead_data =  dict.fromkeys(lead_order)
    #lead_data = {leadid: None for k in lead_order}

#     for all_lead_data in dic['RestingECG']['Waveform']:
#         for single_lead_data in lead['LeadData']:
#             leadname =  single_lead_data['LeadID']
#             if leadname in (lead_order):

    for lead in dic['RestingECG']['Waveform']:
        for leadid in range(len(lead['LeadData'])):
                sample_length = len(decode_ekg_muse_to_array(lead['LeadData'][leadid]['WaveFormData']))
                #sample_length is equivalent to dic['RestingECG']['Waveform']['LeadData']['LeadSampleCountTotal']
                if sample_length == 5000 or sample_length == 5500:
                    lead_data[lead['LeadData'][leadid]['LeadID']] = decode_ekg_muse_to_array(lead['LeadData'][leadid]['WaveFormData'], downsample = 1)
                elif sample_length == 2500:
                    raise ValueError("Sample frequency too low, try with frequency = 500/s")
                else:
                    continue
            #ensures all leads have 2500 samples and also passes over the 3 second waveform
    lead_data['III'] = (np.array(lead_data["II"]) - np.array(lead_data["I"]))
    lead_data['aVR'] = -(np.array(lead_data["I"]) + np.array(lead_data["II"]))/2
    lead_data['aVF'] = (np.array(lead_data["II"]) - np.array(lead_data["I"]))/2
    lead_data['aVL'] = (np.array(lead_data["I"]) - np.array(lead_data["II"]))/2
    
    lead_data = {k: lead_data[k] for k in lead_order}
    # drops V3R, V4R, and V7 if it was a 15-lead ECG

    # now construct and reshape the array
    # converting the dictionary to an np.array
    temp = []
    for key,value in lead_data.items():
        temp.append(value)

    #transpose to be [time, leads, ]
    ekg_array = np.array(temp).T

    #expand dims to [time, leads, 1]
    ekg_array = np.expand_dims(ekg_array,  axis=-1)

    # Here is a check to make sure all the model inputs are the right shape
#     assert ekg_array.shape == (2500, 12, 1), "ekg_array is shape {} not (2500, 12, 1)".format(ekg_array.shape )

    # filename = '/ekg_waveform_{}_{}.npy'.format(pt_id, requisition_number)
    filename = '{}_{}_{}.npy'.format(pt_id, AcquisitionDateTime,PharmaUniqueECGID)
    path_to_output = os.path.join(path_to_output,filename)
    # print(path_to_output)
    if len(ekg_array.shape)==3:
        ekg_array = ekg_array[:,:,0]
    # if len(ekg_array) == 5500:
    #     ekg_array = ekg_array[:-500,:]
    with open(path_to_output, 'wb') as f:
        np.save(f, ekg_array)
    if not type(df) == type(pd.DataFrame()):
        col = list(dic['RestingECG']['PatientDemographics'].keys())
        col.append("Filename")
        col.append('LocationName')
        col.append('SiteName')
        for key in dic['RestingECG']['RestingECGMeasurements'].keys():
            col.append(key)
        col.append('DiagnosisStatement')
        col.append('TestReason')
        col.append('AdmitDiagnosis')
        col.append('xmlFilename')
        df = pd.DataFrame(columns=col)
    pid = dic['RestingECG']['PatientDemographics']
    pid['Filename']=filename
    pid['LocationName']=dic['RestingECG']['TestDemographics']['LocationName']
    pid['SiteName']= dic['RestingECG']['TestDemographics']['SiteName']
    for key in dic['RestingECG']['RestingECGMeasurements'].keys():
        pid[key]=dic['RestingECG']['RestingECGMeasurements'][key]
    
    try:
        pid['AdmitDiagnosis'] = dic['RestingECG']['Order']['AdmitDiagnosis']
    except:
        pid['AdmitDiagnosis'] = ''
    try:
        pid['TestReason'] = dic['RestingECG']['TestDemographics']['TestReason']
    except:
        pid['TestReason'] = ''
    
    try:
        s = ''
        for sen in dic['RestingECG']['Diagnosis']['DiagnosisStatement']:
            s+=sen['StmtText']
        pid['DiagnosisStatement'] = s
    except:
        pid['DiagnosisStatement'] = ''
    pid['xmlFilename'] = path_to_xml.split('/')[-1]
    df=df.append(pid,ignore_index=True)
    return df

def ekg_batch_run(ekg_list,args):
    i = 0
    x = 0
    df = 1
    for file in tqdm(ekg_list):
        try:
            df = xml_to_np_array_file(os.path.join(args['in_XML'],file), args['in_dataset'],df=df)
            i+=1
        except Exception as e:

            print("file failed: ", file)
            print(file, e)
            x+=1
        if i % 10000 == 9999:
            print(f"Succesfully converted {i} EKGs, failed converting {x} EKGs")
            df.to_csv(os.path.join(args['in_dataset'],'Personal_Data.csv'),index=False)
    df.to_csv(os.path.join(args['in_dataset'],'Personal_Data.csv'),index=False)
    print(f"Succesfully converted {i} EKGs, failed converting {x} EKGs")

if not args['in_XML'] is None:
    ekg_file_list = []
    ekg_file_list = file_path(args['in_XML'])  #if you want input to be a directory
    print("Number of EKGs found: ", len(ekg_file_list))

    ekg_batch_run(ekg_file_list,args)




In [ ]:
def std(n1,s1,n2,s2,y1,y2,y):
    top = n1*(s1**2)+n1*(s1**2) + n1*((y1-y)**2)+n2*((y2-y)**2)
    bot = n1+n2
    return np.sqrt(top/bot)
def calc_mean_std(arr,mean,stds,count):
    count_t = count+1
    arr_mean = np.mean(arr,axis=0)
    arr_std = np.std(arr,axis=0)
    new_mean = arr_mean*1/count_t + mean * (count_t-1)/count_t
    return new_mean,std(5000,arr_std,5000*count,stds,arr_mean,mean,new_mean)


fnames = pd.read_csv(args['in_csv']).Filename
out = args['out_dataset']
input_dir = args['in_dataset']
trainData = []

cur_mean = None
cur_std = None

arr = np.zeros(12)
count = 1
if args['mean'] is None:
    for npfile in tqdm(np.random.choice(fnames,args['sample'])):
        path = os.path.join(input_dir,npfile)
        file = np.load(path)
        trainData.append(file)

    trainData = np.array([trainData])
    m = []

    s = []
    for i in tqdm(range(0,12)):
        m.append(np.mean(trainData[:,:,:,i]))
        s.append(np.std(trainData[:,:,:,i],axis=None))
else:
    print("Existing mean and std")
    m = args['mean']
    s = args['std']
del trainData
for i in tqdm(fnames):
    if not os.path.exists(os.path.join(out,i)):
        file = np.load(os.path.join(input_dir,i))
        for k in range(0,12):
            file[:,k] = (file[:,k] - m[k])/s[k]

        np.save(os.path.join(out,i),file)
print("Finished")
